### Imports & Definitions

In [3]:
import sys
sys.path.append("../../utils")
from definitions import *
from parsing_helpers import get_num_diplomas

In [2]:
years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016]

### Convert .doc, .docx & .pdf into .txt

In [54]:
def convert_works_to_txt(year):
    base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
    base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

    processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
    processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)

    def convert_doc_to_txt(filename):
        output_file = processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt")
        if not output_file.is_file():
            subprocess.run(["textutil", "-convert", "txt", "-output", output_file.as_posix(), filename.as_posix()])

    def convert_pdf_to_txt(filename):
        output_file = processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt")
        if not output_file.is_file():
            subprocess.run(["pdftotext", filename.as_posix(), output_file.as_posix()])

    for ext in [".doc", ".docx", ".DOCX"]:
        for file in tqdm(base_diplomas_dir_path.glob(f"*{ext}"), desc="Docs..."):
            convert_doc_to_txt(file)

    
    for file in tqdm(base_diplomas_dir_path.glob(f"*.pdf"), desc="Pdfs..."):
        convert_pdf_to_txt(file)

In [55]:
for year in tqdm(years, desc="Years..."):
    convert_works_to_txt(year)

Docs...: 29it [00:00, 5552.58it/s]:00<?, ?it/s]
Error reading /Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/2023/work/44850_diploma.docx.  The file “44850_diploma.docx” couldn’t be opened.
Error reading /Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/2023/work/39877_diploma.docx.  The file isn’t in the correct format.
Docs...: 722it [00:00, 2865.11it/s]
Docs...: 1it [00:00, 329.40it/s]
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Pdfs...: 881it [00:00, 3019.44it/s]
Docs...: 25it [00:00, 3663.53it/s]:00<00:03,  1.79it/s]
Docs...: 708it [00:00, 51967.30it/s]
Docs...: 1it [00:

### Making df

In [57]:
base_diplomas_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/")
new_rows = []
for year in tqdm(years, desc="Years..."):
    ids_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/ids.json")
    skipped_ids_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/ids.json")
    with open(ids_path, "r") as f:
        ids = json.load(f)
    with open(skipped_ids_path, "r") as f:
        skipped_ids = set(json.load(f))
    new_rows_year = []
    for id in ids:
        diploma_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/work_txt/{id}_diploma.txt")
        abstract_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/work/{id}_abstract.txt")
        if not abstract_path.is_file() or not diploma_path.is_file():
            assert not diploma_path.is_file(), f"Strange for id = {id}"
            assert id in skipped_ids
            continue
        with open(diploma_path, "r") as f:
            diploma = f.read()
        with open(abstract_path, "r") as f:
            abstract = f.read()
        new_rows_year.append({
            "id": id,
            "year": year,
            "diploma": diploma,
            "abstract": abstract,
        })
    df = pd.DataFrame(new_rows_year)
    df.to_csv(base_diplomas_spbu_dir_path.joinpath(f"{year}/dataset.csv"), index=False)
    new_rows.extend(new_rows_year)
df = pd.DataFrame(new_rows)
df.to_csv(base_diplomas_spbu_dir_path.joinpath(f"dataset.csv"), index=False)
df.head()

Years...: 100%|██████████| 8/8 [00:58<00:00,  7.29s/it]


,id,year,diploma,abstract
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...


### View russian diplomas

In [6]:
df_2023 = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/russian_stupid_filter_dataset.csv"))

In [7]:
print(len(df_2023))
df_2023 = df_2023[df_2023["year"] == 2023]
len(df_2023)

10924


1387

### Calculating total capacity of diplomas

In [4]:
total = 0
for year in years:
    total += get_num_diplomas(year)
total

17137

### Filter df: remain russian (stupid)

In [69]:
pattern = "выпускная квалификационная работа"
lower_texts = list(map(lambda x: x.lower(), df["diploma"]))
mask = []
for lower_text in lower_texts:
    mask.append(pattern in lower_text)

In [70]:
sum(mask), len(mask), sum(mask) / len(mask) * 100

(10924, 14398, 75.87164884011668)

In [79]:
np.where(mask)[0]

array([    1,     2,     3, ..., 14392, 14394, 14395])

In [80]:
russian_stupid_filter_df = df.iloc[np.where(mask)[0]]

In [83]:
russian_stupid_filter_df.to_csv(base_diplomas_spbu_dir_path.joinpath(f"russian_stupid_filter_dataset.csv"), index=False)

### [old] Convert .doc to .txt

In [4]:
base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

exts = defaultdict(int)
for file in base_diplomas_dir_path.glob("*"):
    exts[file.suffix] += 1
print(exts)

for ext in exts:
    for file in base_diplomas_dir_path.glob(f"*{ext}"):
        if ext != ".pages":
            break

processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)
def convert_doc_to_txt(filename):
    subprocess.run(["textutil", "-convert", "txt", "-output", processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix(), filename])

# for ext in [".doc", ".docx", ".DOCX"]:
#     for file in base_diplomas_dir_path.glob(f"*{ext}"):
#         convert_doc_to_txt(file)

defaultdict(<class 'int'>, {'.txt': 1654, '.pdf': 881, '.docx': 722, '.doc': 29, '.jpg': 4, '.spv': 1, '.jpeg': 7, '.pages': 1, '.mp4': 1, '.DOCX': 1, '.xlsx': 2, '.rtf': 1, '.odt': 1, '.docm': 1, '.JPG': 1, '.PDF': 1})


In [8]:
def convert_pdf_to_txt(filename):
    subprocess.run(["pdftotext", filename.as_posix(), processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix()])

for file in base_diplomas_dir_path.glob(f"*.pdf"):
    convert_pdf_to_txt(file)

Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Error: Document stream is empty
Syntax Warning: Invalid Font Weight
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warnin

### Filter diplomas: only russian

In [3]:
RAW_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/work/")
PROCESSED_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/stupid_filtered_work")
PROCESSED_DIPLOMAS_DIR_PATH.mkdir(exist_ok=True, parents=True)

In [3]:
num_ok, num_all = 0, 0
for file in RAW_DIPLOMAS_DIR_PATH.glob("*.txt"):
    with open(file, "r") as f:
        text = f.read()
    if "выпускная квалификационная работа" in text.lower() and "отзыв" not in text.lower():
        with open(PROCESSED_DIPLOMAS_DIR_PATH.joinpath(f"{file.stem}.txt"), "w") as f:
            f.write(text)
        print(text[:10])
        num_ok += 1
    num_all += 1
print(num_ok, num_all)

Санкт-Пете
          
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
ФГОУ ВО «С
Санкт-Пете
Санкт-Пете
Санкт-Пете
САНКТ-ПЕТЕ
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете

САНКТ-ПЕТ
Санкт-Пете
Санкт-Пете
23 143
